In [1]:
!pip install tensorflow_text
!pip install tensorflow_hub

  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.3 MB)
Using cached tensorboard-2.17.1-py3-none-any.whl (5.5 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.2
    Uninstalling tensorflow-2.16.2:
      Successfully uninstalled tensorflow-2.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.17.0 which is incompatible.
  Using cached tensorflow-2.16.2-cp310

In [2]:
!pip install tf-keras==2.16

In [3]:
 import os
 os.environ['TF_USE_LEGACY_KERAS']='1'

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_datasets as tfds
import datetime

In [6]:
# Load the IMDB dataset using TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True, with_info=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.8KXWKZ_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.8KXWKZ_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.8KXWKZ_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [7]:
# Split the training data into train and validation sets
def train_val_split(ds, val_size=0.2):
    ds = ds.shuffle(10000, seed=42)
    val_size = int(val_size * ds_info.splits['train'].num_examples)
    ds_val = ds.take(val_size)
    ds_train = ds.skip(val_size)
    return ds_train, ds_val

ds_train, ds_val = train_val_split(ds_train)


In [8]:
# Reduce batch size to lessen memory usage
batch_size = 32
ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [9]:
# Load the preprocessor and ALBERT model from TensorFlow Hub
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/3", trainable=True)


In [10]:
# Build the model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocessor(text_input)
encoder_outputs = encoder(preprocessed_text)
pooled_output = encoder_outputs['pooled_output']
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(pooled_output)


In [11]:
model = tf.keras.Model(inputs=[text_input], outputs=[output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Enable mixed precision (if GPU supports it)
#from tensorflow.keras.mixed_precision import set_global_policy
#policy = tf.keras.mixed_precision.Policy('mixed_float16')
#set_global_policy(policy)


In [12]:
# Train the model
epochs = 3
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [13]:
model.fit(ds_train, validation_data=ds_val, epochs=epochs, callbacks=[early_stopping])

Epoch 1/3
625/625 [==============================] - 2537s 4s/step - loss: 0.3437 - accuracy: 0.8474 - val_loss: 0.2425 - val_accuracy: 0.8980
Epoch 2/3
625/625 [==============================] - 2425s 4s/step - loss: 0.2437 - accuracy: 0.9004 - val_loss: 0.2094 - val_accuracy: 0.9188
Epoch 3/3
625/625 [==============================] - 2413s 4s/step - loss: 0.1868 - accuracy: 0.9270 - val_loss: 0.1971 - val_accuracy: 0.9256


In [14]:
# Evaluate the model
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy:.4f}")

782/782 [==============================] - 842s 1s/step - loss: 0.3154 - accuracy: 0.8733
Test Accuracy: 0.8733
